In [118]:
from pathlib import Path

import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
from sklearn.model_selection import train_test_split

import pyro
from pyro.distributions import Categorical, Bernoulli
from pyro import param
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import MCMC, NUTS
from pyro.distributions import constraints
import pyro.distributions as dist
from pyro.infer import Predictive

In [32]:
feature_path = Path.cwd().parent / 'train_test_features.parquet'
data = pd.read_parquet(feature_path) 
train, test = train_test_split(data, test_size=0.2) 

In [86]:
def discretize(data):
    '''
    transforms continuous data in the range [0,1] and
    no data values of  -99 to integers from 0 to 11.
    Where 11 is no data value'''
    # Handle the special case
    special_case = (data == -99).int() * 11
    discretized = (torch.clamp(torch.round(data * 10).int(), 0, 10) + special_case).long()
    return discretized

ahh

In [87]:
# features, found in mortality_classification.ipynb
# based on abs(corr) < 0.4, and RF feature importance
feature_names = [
    'n_mean',
    'lum70',
    'savi_std',
    'rgi_std',
    'lum50',
    'lum30',
    'b_std',
    'r_std',
    'r40',
    'lum40',
    'g30',
    'b50',
    'lum10',
    'b10',
    'n10',
    'rgi60',
    'rgi80',
    'r60',
    'b80',
    'rgi30',
    'r70',
    'n80',
    'b60'
 ]

features = torch.tensor(
    train[feature_names].values,
    dtype=torch.float32
)

labels = torch.tensor(
    train.y.values,
    dtype=torch.long
    )


In [137]:
# define model
def model(features, p, labels=None):
    n_samples, n_features = features.shape
    
    # If p_prior is a scalar, we replicate it to match the batch shape.
    probs = p * torch.ones(n_samples, dtype=torch.float32)
    
    with pyro.plate('data', size=n_samples):
        # capture samples of p for the posterior
        pyro.sample('p_aux', dist.Delta(probs), obs=probs)
        
        if labels is not None:
            y = pyro.sample('y', Bernoulli(probs), obs=labels.float())
        else:
            y = pyro.sample('y', Bernoulli(probs))
        
        for i in range(n_features):
            discretized_feature = discretize(features[:, i])
            pyro.sample(f'obs_{i}', Categorical(probs=torch.ones(12)/12), obs=discretized_feature)


# make a nn guide class
class NNGuide(nn.Module):
    def __init__(self, input_size, hidden_size=10):
        super(NNGuide, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
    
    def forward(self, features):
        x = torch.relu(self.fc1(features))
        return torch.sigmoid(self.fc2(x))


# instantiate nn guide
guide_model = NNGuide(23)  # Assuming 23 features


# def model guide
def guide(features, p, labels=None):
    estimated_p = guide_model(features).squeeze(-1)

    with pyro.plate('data', size=features.shape[0]):
        pyro.sample('y', dist.Bernoulli(estimated_p))
        pyro.sample('p_aux', dist.Delta(estimated_p))

# train model and guide using SVI
optimizer = Adam({'lr': 0.01})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())


# prior based on labels
p_prior = float(train.y.sum() / len(train.y))

labels_reshaped = labels.unsqueeze(-1)

num_epochs = 5000 

for epoch in range(num_epochs):
    loss = svi.step(features.float(), p_prior, labels.long())
    if epoch % 100 == 0:  # print loss every 100 steps
        print(f'Epoch {epoch} Loss: {loss}')



Epoch 0 Loss: 576205.5454101562
Epoch 100 Loss: 576202.1416015625
Epoch 200 Loss: 576197.7280273438
Epoch 300 Loss: 576203.0209960938
Epoch 400 Loss: 576202.9204101562
Epoch 500 Loss: 576196.40234375
Epoch 600 Loss: 576204.3061523438
Epoch 700 Loss: 576207.6064453125
Epoch 800 Loss: 576205.3486328125
Epoch 900 Loss: 576201.6591796875
Epoch 1000 Loss: 576206.0385742188
Epoch 1100 Loss: 576201.205078125
Epoch 1200 Loss: 576199.0737304688
Epoch 1300 Loss: 576204.3315429688
Epoch 1400 Loss: 576201.1235351562
Epoch 1500 Loss: 576201.2734375
Epoch 1600 Loss: 576203.302734375
Epoch 1700 Loss: 576205.607421875
Epoch 1800 Loss: 576196.8564453125
Epoch 1900 Loss: 576197.5126953125
Epoch 2000 Loss: 576200.8984375
Epoch 2100 Loss: 576192.80078125
Epoch 2200 Loss: 576209.6684570312
Epoch 2300 Loss: 576211.6010742188
Epoch 2400 Loss: 576208.1000976562
Epoch 2500 Loss: 576209.5830078125
Epoch 2600 Loss: 576203.9321289062
Epoch 2700 Loss: 576198.787109375
Epoch 2800 Loss: 576207.2626953125
Epoch 2900 

In [115]:
test_features = torch.tensor(
    test[feature_names].values,
    dtype=torch.float32
)

test_labels = torch.tensor(
    test.y.values,
    dtype=torch.long
    )

In [135]:
# pass the new data through the trained guide model
with torch.no_grad():
    p_posterior_mean = guide_model(test_features).numpy()

In [143]:
num_samples = 1000

predictive = Predictive(model, guide=guide, num_samples=num_samples, return_sites=['y', 'p_aux'])

samples = predictive(test_features, p_prior)

In [144]:
samples['p_aux']

tensor([[0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        ...,
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693]])

In [139]:
y_samp = samples['p_aux']
y_samp

KeyError: 'p_aux'

tensor(0.5310)